In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras
import tensorflow
from sklearn import model_selection

Using TensorFlow backend.


In [6]:
data = pd.read_csv('data.csv')

In [7]:
data.drop(columns=['Unnamed: 0'], inplace = True)

In [8]:
y = data.iloc[:, -1]

In [9]:
X = data.iloc[:, :data.shape[1] - 1]

In [47]:
cnn_par = {
    'window': [3, 7, 11],
    'dropout': [0.1, 0.2, 0.5],
    'epochs': [25]
}

In [48]:
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import GlobalAveragePooling1D
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier

In [76]:
X=np.array(X)

In [80]:
X = X.reshape(X.shape[0], X.shape[1], 1)
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.33, stratify = y)

In [81]:
from sklearn import preprocessing

In [82]:
#scaler_train = preprocessing.StandardScaler()
#scaler_train.fit(X_train)
#X_train_cnn = scaler_train.transform(X_train)
#X_test_cnn = scaler_train.transform(X_test)

In [83]:
filter_cnn = 64
number_of_classes = len(np.unique(y))
input_shape = (X_train.shape[1], 1)

In [84]:
#from keras import utils
#y_train = utils.to_categorical(y_train, number_of_classes)
#y_test = utils.to_categorical(y_test, number_of_classes)

In [85]:
pool = 3
num_classes = 5

In [86]:
#X_train = np.expand_dims(np.random.normal(size=(7705, 178)),axis=-1)

In [87]:
def mreza(input_shape, num_classes,window, dropout):
    model = Sequential()
    model.add(Conv1D(filter_cnn, window, activation='relu', input_shape=input_shape))
    model.add(Conv1D(filter_cnn, window, activation='relu'))
    model.add(MaxPooling1D(pool))
    model.add(Conv1D(filter_cnn * 2, window, activation='relu'))
    model.add(Conv1D(filter_cnn * 2, window, activation='relu'))
    model.add(GlobalAveragePooling1D())
    model.add(Dropout(dropout))
    model.add(Dense(number_of_classes, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [88]:
from keras.wrappers.scikit_learn  import KerasClassifier
def get_estimator(input_shape=input_shape, num_classes=num_classes,window=3, dropout=0.5):
    return KerasClassifier(build_fn=mreza,input_shape=input_shape, num_classes=num_classes,window=window, dropout=dropout)

In [89]:
estimator = get_estimator()

In [90]:
grid_cnn = model_selection.GridSearchCV(estimator, param_grid=cnn_par, scoring='accuracy', cv=10, return_train_score=True)

In [ ]:
grid_cnn.fit(X_train, y_train)

Epoch 1/25
6934/6934 [==============================] - 11s 2ms/step - loss: 1.3466 - accuracy: 0.4616
Epoch 2/25
6934/6934 [==============================] - 11s 2ms/step - loss: 0.8546 - accuracy: 0.6273
Epoch 3/25
6934/6934 [==============================] - 11s 2ms/step - loss: 0.7207 - accuracy: 0.6742
Epoch 4/25
6934/6934 [==============================] - 11s 2ms/step - loss: 0.6395 - accuracy: 0.6992 
Epoch 5/25
6934/6934 [==============================] - 11s 2ms/step - loss: 0.6122 - accuracy: 0.7120
Epoch 6/25
6934/6934 [==============================] - 11s 2ms/step - loss: 0.5864 - accuracy: 0.7240
Epoch 7/25
6934/6934 [==============================] - 11s 2ms/step - loss: 0.5614 - accuracy: 0.7342
Epoch 8/25
1344/6934 [====>.........................] - ETA: 8s - loss: 0.5383 - accuracy: 0.7426

In [ ]:
grid_cnn.best_score_